In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [ ]:
df=pd.read_excel('Dataset.xlsx',sheet_name='E Comm')
df.head()

In [ ]:
df.shape

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
for i in df.columns:
    if df[i].dtypes == 'object':
        print(i)
        print()
        print('the values are:') 
        print(df[i].value_counts())
        print()
        print()

In [ ]:
df1=df.copy()  


In [ ]:
df.drop(['CustomerID'],axis=1, inplace=True)

In [ ]:
for i in df.columns:
    if df[i].isnull().sum() > 0:
        print(i)
        print('the total null values are:', df[i].isnull().sum())
        print('the datatype is', df[i].dtypes)
        print()

In [ ]:
df['Churn'] = df['Churn'].astype('object')
df['CityTier'] = df['CityTier'].astype('object')

In [ ]:
df.describe().transpose()

In [ ]:
for i in df.columns:
    if df[i].isnull().sum() > 0:
        df[i]=df[i].fillna(df[i].median())
df.isnull().sum()
df.head()

## Treating outliers

In [ ]:
plt.figure(figsize=(50,10))
sns.boxplot(data=df)
plt.title('The boxplot to study outliers')
plt.xlabel('Variables that predict the customer churn')
plt.ylabel('Values')

after checking outliers now we remove those outliers

In [ ]:
def remove_outlier(col):
    sorted(col)
    Q1,Q3=np.percentile(col,[25,75])
    IQR=Q3-Q1
    lr= Q1-(1.5 * IQR)
    ur= Q3+(1.5 * IQR)
    return lr, ur

df.columns

In [ ]:
for column in df.columns:
    if df[column].dtype !='object':
        lr,ur=remove_outlier(df[column])
        df[column]=np.where(df[column]>ur,ur,df[column])
        df[column]=np.where(df[column]<lr,lr,df[column])

Now plotting again 

In [ ]:
plt.figure(figsize=(50,10))
sns.boxplot(data=df)
plt.title('The boxplot to study outliers')
plt.xlabel('Variables that predict the customer churn')
plt.ylabel('Values')

##Adding a new variable 
cashback per order -> cashbackamout+ordercount

In [ ]:
df['avg_cashbk_per_order']=df['CashbackAmount']/df['OrderCount']

In [ ]:
Churn_perc = round((df['Churn'][df['Churn']==1].count()*100/df['Churn'][df['Churn']==0].count()),2)
print(Churn_perc)


## Data Analysis 
1) univariate analysis

In [ ]:
cat=[]
num=[]
for i in df.columns:
  if df[i].dtype=='object':
    cat.append(i)
  else:
    num.append(i)
print('cat = ',cat)
print('num = ',num)

In [ ]:
df[cat].describe().T

In [ ]:
for i in cat:
    print(i)
    print()
    print(df[i].value_counts())
    print()
    print()

In [ ]:
df[num].describe().T

In [ ]:
df[num].hist(figsize=(40,40))

In [ ]:
fig, ax = plt.subplots(8, 2, figsize=(40, 40))
for i, subplot in zip(num, ax.flatten()):
    sns.histplot(df[i], ax=subplot)

##Analysing churn by each variable

In [ ]:
df.head()

In [ ]:
d = {}

for i in df.columns:
    # Group by the column and get the sum of 'Churn', renaming it to 'Customers_churned'
    churn_sum = df.groupby(i).Churn.sum().rename('Customers_churned')
    
    # Get value counts for the column
    value_counts = df[i].value_counts().rename('Total_Customers')
    
    # Calculate percentage (ensure both are aligned first)
    perc_of_total_cust = (churn_sum * 100 / value_counts).round(2).rename('perc_of_total_cust')
    
    # Combine them into a single DataFrame
    temp_df = pd.concat([churn_sum, value_counts, perc_of_total_cust], axis=1)
    
    # Reset the index and rename the index column
    temp_df.reset_index(level=0, inplace=True)
    temp_df = temp_df.rename(columns={ 'index': i })
    
    # Store in the dictionary
    d[i] = temp_df


In [ ]:
for i in df.columns:
    print(i)
    print(d[i])
    print()

## Analysing churn by each variable --by visualising via graphs

In [ ]:
def analysis_chart(variable):
  # definig the plot for matplotlib
  plt.figure(figsize=(20,12))
  fig, ax = plt.subplots()
  # defining the title
  title1 = 'Customers Churn analysed by ' + variable
  plt.title(title1)
  # defining the lines for the y -axis
  line1 = ax.plot(d[variable][variable],d[variable]['Customers_churned'], color='lightskyblue', label = 'Customers churned')
  line2 = ax.plot(d[variable][variable],d[variable]['Total_Customers'], color='dodgerblue', label = 'Total Customers')
  # labelling the x -axis and y-axis
  plt.xlabel (variable)
  plt.ylabel ('No. of customers')
  # rotating the labels on the x-axis for better visualisation
  for tick in ax.get_xticklabels():
    tick.set_rotation(45)
  # defining another axis on the right side of the graph
  ax2=ax.twinx()
  # defining the line for the right side y -axis
  line3 = ax2.plot(d[variable][variable],d[variable]['perc_of_total_cust'], color='yellowgreen', label = 'Churn as Percent of total')
  y = 0*d[variable]['perc_of_total_cust']+20.25
  line4 = ax2.plot(d[variable][variable], y, color='orangered', label='Average customer Churn', linestyle='dashed')
  # labelling the right side y-axis
  plt.ylabel ('percentage of customers churned')
  # adding the three lines to show the legend on the right corner in a coherent place, not doing this will lead to overlapping of legends of lines belonging to left and right y axis
  lines = line1+line2+line3+line4
  labs = [l.get_label() for l in lines]
  ax.legend(lines, labs, bbox_to_anchor=(1.7, 1))
  # adding sns palette for better visualisation
  sns.despine(ax=ax, right=True, left=True)
  sns.despine(ax=ax2, left=True, right=False)

In [ ]:
col = ['Tenure', 'PreferredLoginDevice', 'CityTier',
       'WarehouseToHome', 'PreferredPaymentMode', 'Gender', 'HourSpendOnApp',
       'NumberOfDeviceRegistered', 'PreferedOrderCat', 'SatisfactionScore',
       'MaritalStatus', 'NumberOfAddress', 'Complain',
       'OrderAmountHikeFromlastYear', 'CouponUsed', 'OrderCount',
       'DaySinceLastOrder']

In [ ]:
analysis_chart('Tenure')

In [ ]:
analysis_chart('CityTier')


In [ ]:
analysis_chart('WarehouseToHome')

In [ ]:
analysis_chart('PreferredPaymentMode')

In [ ]:
analysis_chart('Gender')


In [ ]:
analysis_chart('HourSpendOnApp')


In [ ]:
analysis_chart('NumberOfDeviceRegistered')

In [ ]:
analysis_chart('PreferedOrderCat')
analysis_chart('SatisfactionScore')
analysis_chart('MaritalStatus')
analysis_chart('NumberOfAddress')
analysis_chart('Complain')
analysis_chart('OrderAmountHikeFromlastYear')
analysis_chart('CouponUsed')
analysis_chart('OrderCount')
analysis_chart('DaySinceLastOrder')


In [ ]:
# writing the loop for automating the figure generation
# Writing the loop for one dataframe at a time since we need to have the figures in separate cells so that it becomes easy to analyse and write the conclusion there only.
for i in ['avg_cashbk_per_order']:
    # definig the plot for matplotlib
    plt.figure(figsize=(20,12))
    fig, ax = plt.subplots()
    # defining the title
    title1 = 'Customers Churn analysed by ' + i
    plt.title(title1)
    # defining the lines for the y -axis
    line1 = ax.scatter(d[i][i],d[i]['Customers_churned'], color='lightskyblue', label = 'Customers churned')
    line2 = ax.scatter(d[i][i],d[i]['Total_Customers'], color='dodgerblue', label = 'Total Customers')
    # labelling the x -axis and y-axis
    plt.xlabel (i)
    plt.ylabel ('No. of customers')
    # rotating the labels on the x-axis for better visualisation
    for tick in ax.get_xticklabels():
      tick.set_rotation(45)
    # defining another axis on the right side of the graph
    ax2=ax.twinx()
    # defining the line for the right side y -axis
    line3 = ax2.scatter(d[i][i],d[i]['perc_of_total_cust'], color='yellowgreen', label = 'Churn as Percent of total')
    # labelling the right side y-axis
    plt.ylabel ('percentage of customers churned')
    # adding sns palette for better visualisation
    sns.despine(ax=ax, right=True, left=True)
    sns.despine(ax=ax2, left=True, right=False)

# Bivariate analysis (some error currently)

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(df.corr(),annot=True)

# One hot encoding and Scaling of data

In [ ]:
df.head()


In [ ]:
df_encoded=df.copy()
df_encoded.head()

In [ ]:
df_encoded = pd.get_dummies(df_encoded,drop_first=True)

In [119]:
scaler = StandardScaler()


In [120]:
features = df_encoded[num]
features = scaler.fit_transform(features)

In [121]:
scaled_df_encoded = df_encoded.copy()


In [122]:
scaled_df_encoded[num] = features


In [123]:
scaled_df_encoded


,Tenure,WarehouseToHome,HourSpendOnApp,NumberOfDeviceRegistered,SatisfactionScore,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,...,PreferredPaymentMode_E wallet,PreferredPaymentMode_UPI,Gender_Male,PreferedOrderCat_Grocery,PreferedOrderCat_Laptop & Accessory,PreferedOrderCat_Mobile,PreferedOrderCat_Mobile Phone,PreferedOrderCat_Others,MaritalStatus_Married,MaritalStatus_Single
0,-0.737959,-1.178795,0.092878,-0.738345,-0.772992,1.875626,1.584290,-1.304023,-0.411136,-0.888745,...,False,False,False,False,True,False,False,False,False,True
1,-0.134866,-0.931534,0.092878,0.323303,-0.048392,1.092812,1.584290,-0.187486,-1.344668,-0.888745,...,False,True,True,False,False,True,False,False,False,True
2,-0.134866,1.788340,-1.328053,0.323303,-0.048392,0.701405,1.584290,-0.466620,-1.344668,-0.888745,...,False,False,True,False,False,True,False,False,False,True
3,-1.220433,-0.066119,-1.328053,0.323303,1.400807,1.484219,-0.631198,2.045589,-1.344668,-0.888745,...,False,False,True,False,True,False,False,False,False,True
4,-1.220433,-0.437011,0.092878,-0.738345,1.400807,-0.472817,-0.631198,-1.304023,-0.411136,-0.888745,...,False,False,True,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5625,-0.014247,1.788340,0.092878,-1.799993,-1.497592,0.701405,-0.631198,0.649917,-0.411136,-0.304276,...,False,False,True,False,True,False,False,False,True,False
5626,0.347609,-0.313381,0.092878,1.384951,1.400807,0.701405,-0.631198,0.091648,-0.411136,-0.304276,...,False,False,True,False,False,False,False,False,True,False
5627,-1.099814,-0.560642,0.092878,-1.799993,0.676207,-0.472817,1.584290,1.487320,-0.411136,-0.304276,...,False,False,True,False,True,False,False,False,True,False
5628,1.553794,-0.807903,1.513809,1.384951,0.676207,-0.081410,-0.631198,-0.187486,0.522397,-0.304276,...,False,False,True,False,True,False,False,False,True,False


# Now our data is preprocessed and ready to be used for training our model. 